In [ ]:
#@title import 
import numpy as np


In [ ]:
#@title load data
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
#@title Compute entropy
def compute_entropy(y):
      entropy = 0.
    
      if len(y)==0:
          entropy=0
      else:
          count=0
          for i in y:
              if i==1:
                  count+=1
          p1=count/len(y)
          if p1==0 or p1==1:
              return 0
          entropy=(-p1*np.log2(p1))-((1-p1)*np.log2(1-p1))
            
      return entropy

In [ ]:
#@title split data
def split_dataset(X, node_indices, feature):

    
    left_indices = []
    right_indices = []
    n,m=X.shape
    for i in range(n):
        if i in node_indices:
            if X[i][feature]==1:
                left_indices.append(i)
            else:
                right_indices.append(i)
        
    return left_indices, right_indices

In [ ]:
#@title Calculate IG
def compute_information_gain(X, y, node_indices, feature):
    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
   
    information_gain = 0
    wl=len(y_left)/len(y_node)
    wr=len(y_right)/len(y_node)
    
    
    information_gain= compute_entropy(y_node)-((wl*compute_entropy(y_left))+(wr*compute_entropy(y_right)))
  
    return information_gain

In [ ]:
#@title Find best feature
def get_best_split(X, y, node_indices):   
    
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    is_pure=True
    one_present=False
    zero_present=False
    for i in y:
        if i==1:
            one_present=True
        if i==0:
            zero_present=True
    if one_present and zero_present:
        is_pure=False
    if len(node_indices)<=1 or is_pure:
        return best_feature
    else:
        info_gain =[]
        for i in range (num_features):
            info_gain.append((compute_information_gain(X, y, node_indices, i)))

        best_infp_gain = max(info_gain)
        best_feature = info_gain.index(best_infp_gain)
        
    
   
    return best_feature

In [ ]:
#@title Create tree
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [ ]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
